In [1]:
from cobra.flux_analysis import pfba
from os import getcwd
import math
import numpy as np
import pandas as pd
import holoviews as hv
from __future__ import unicode_literals
import matplotlib

In [2]:
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite, Model

In [3]:
matplotlib.interactive(True)

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
relative_directory = getcwd()
mcaps = read_sbml_model(relative_directory + '/../ModelFiles/xml/iMcBath.xml')

## Reactions using Urea

In [6]:
mcaps.metabolites.urea_p.reactions

frozenset({<Reaction UREAabcpp at 0x11904b8d0>,
           <Reaction UREAtex at 0x11904b990>})

In [7]:
mcaps.reactions.DADK.gene_reaction_rule

'MCA2066'

In [8]:
for rxn in mcaps.metabolites.urea_c.reactions:
    print rxn.id, rxn.gene_reaction_rule

BLUB MCA2289
UREASE_GAPFILLING 
UREAabcpp MCA1662


In [9]:
mcaps.medium

{'EX_ca2_e': 1000.0,
 'EX_cd2_e': 1000.0,
 'EX_ch4_e': 18.46,
 'EX_cl_e': 1000.0,
 'EX_co_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_n2_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

## Results Collection

In [10]:
df = pd.DataFrame(index=["Methane UTR","Oxygen UTR", "CO2 UTR", "NH4 UTR", "N2 UTR",
                         "NO3 UTR", "Uphill Electron Transfer Flux", "Growth Rate" ,
                         "Growth Yield", "O2/CH4 Ratio", "NH4/CH4 Ratio", "N2/CH4 Ratio",
                         "NO3/CH4 Ratio", "ATP/CH4 Ratio", "CCE"])

In [11]:
def df_record(solution, df, column):
    gr = solution.fluxes.BIOMASS_Mcapsulatus
    bc = carbon_in_biomass(mcaps.reactions.BIOMASS_Mcapsulatus)["C"]
    yx = gr/(abs((solution.fluxes.EX_ch4_e/1000)*16.04))
    data = [solution.fluxes.EX_ch4_e, solution.fluxes.EX_o2_e, solution.fluxes.EX_co2_e, solution.fluxes.EX_nh4_e,
            solution.fluxes.EX_n2_e, solution.fluxes.EX_no3_e, solution.fluxes.CYOR_q8ppi, gr, yx, 
            solution.fluxes.EX_o2_e/solution.fluxes.EX_ch4_e, solution.fluxes.EX_nh4_e/solution.fluxes.EX_ch4_e,
            solution.fluxes.EX_n2_e/solution.fluxes.EX_ch4_e, solution.fluxes.EX_no3_e/solution.fluxes.EX_ch4_e,
            solution.fluxes.ATPSipp/abs(solution.fluxes.EX_ch4_e), ((gr * bc)/ solution.fluxes.EX_ch4_e)*100]
    df[column] = data
    return df

In [12]:
mcaps.reactions.EX_n2_e.bounds = 0, 1000
mcaps.reactions.ATPSNaipp.bounds = 0, 0
mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000

In [13]:
def carbon_in_biomass(biomass_rxn):
    total_biomass_formula = {}
    for met, stoich in biomass_rxn.metabolites.items():
        elements = {key: value*stoich for (key, value) in met.elements.items()}
        for key, value in elements.items():
            if key not in total_biomass_formula:
                total_biomass_formula[key] = value
            else:
                total_biomass_formula[key] += value
    return total_biomass_formula

In [14]:
def highlevel_stoichiometry(solution):
    boundary_rxns = set(mcaps.exchanges) & set(solution.reactions)
    for rxn in boundary_rxns:
        if rxn.flux < 0:
            print([met.name for met in rxn.metabolites], round(rxn.flux, 5))
    for rxn in boundary_rxns:
        if rxn.flux > 0:
            print([met.name for met in rxn.metabolites], round(rxn.flux, 5))

## Redox-Arm Only

Carbon-Limited, NO3, pMMO, Redox-Arm Only

In [15]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO, Redox-Arm Only')

In [16]:
highlevel_stoichiometry(solution)

AttributeError: 'Solution' object has no attribute 'reactions'

Carbon-Limited, NO3, sMMO, Redox-Arm Only

In [17]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, sMMO, Redox-Arm Only')

Carbon-Limited, NH3, pMMO, Redox-Arm Only

In [18]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, Redox-Arm Only')

In [129]:
highlevel_stoichiometry(solution)

(['Co(2+)'], -2e-05)
(['Nickel'], -1e-05)
(['phosphate'], -0.1435)
(['dioxygen'], -27.86102)
(['sulfate'], -0.04393)
(['methane'], -18.46)
(['Ca(2+)'], -0.01818)
(['Cl(-)'], -0.0558)
(['Zinc'], -0.00026)
(['fe2_p'], -0.00159)
(['K(+)'], -0.04593)
(['Na(+)'], -0.01019)
(['Ammonium'], -2.13275)
(['Mg(2+)'], -0.03213)
(['Manganese(2+)'], -0.0)
(['Cu2+'], -0.00039)
(['4-Hydroxy-benzyl alcohol'], 1e-05)
(['H2O'], 33.26359)
(['CO(2)'], 9.69903)
(['doxopa_c'], 0.00028)
(['hydrogen peroxide'], 0.08361)
(['H(+)'], 2.13306)
(['carbon monoxide(1+)'], 1e-05)
(['alpha-ribazole'], 0.00028)


Carbon-Limited, NH3, sMMO, Redox-Arm Only

In [19]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, sMMO, Redox-Arm Only')

## Redox-Arm Parameter Exploration

In [20]:
mcaps.reactions.ATPSipp.metabolites

{<Metabolite adp_c at 0x118999150>: -1.0,
 <Metabolite atp_c at 0x1189e6090>: 1.0,
 <Metabolite h_c at 0x118a51390>: 3.0,
 <Metabolite h_p at 0x118a51490>: -4.0,
 <Metabolite h2o_c at 0x118a95f90>: 1.0,
 <Metabolite pi_c at 0x118b401d0>: -1.0}

In [21]:
for r in np.arange(0,2,0.2):
    with mcaps:
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 0
        mcaps.reactions.ATPSipp.add_metabolites(
            {mcaps.metabolites.adp_c: -1.0, 
             mcaps.metabolites.atp_c: 1.0,
             mcaps.metabolites.h2o_c: 1.0,
             mcaps.metabolites.h_c: (3.0-r),
             mcaps.metabolites.h_p: -(4.0-r), 
             mcaps.metabolites.pi_c: -1.0}, combine=False)
        solution = pfba(mcaps)
#         print(mcaps.reactions.ATPSipp)
#         print(solution.fluxes.ATPSipp)

    df = df_record(solution, df, 'NO3, pMMO, Redox-Arm, H/ATP {}'.format(3-r))

## Direct Coupling Exploration

Carbon-Limited, NO3, pMMODC

In [22]:
for r in np.arange(0,6.5,0.5):
    with mcaps:
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.PMMOipp.bounds = 0, 1000
        mcaps.reactions.PMMODCipp.bounds = 0, 1000
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 0
        c3 = mcaps.problem.Constraint(
                mcaps.reactions.PMMOipp.flux_expression - 
                r* mcaps.reactions.PMMODCipp.flux_expression, lb=0, ub=0, name='c3')
        mcaps.add_cons_vars(c3)
        solution = pfba(mcaps)
        print(solution.fluxes.MEOHDHipp,solution.fluxes.PMMODCipp, solution.fluxes.PMMOipp)

    df = df_record(solution, df, 'NO3, pMMODC, RedEff {}'.format(r))

(18.46, 18.46, 0.0)
(18.46, 12.306666666666668, 6.153333333333333)
(18.45999963687165, 9.229999818435825, 9.229999818435823)
(18.46, 7.384, 11.076)
(18.46, 6.153333333333334, 12.306666666666667)
(18.459999999999987, 5.274285714285712, 13.185714285714274)
(18.45999999999965, 4.614999999999913, 13.844999999999736)
(18.459999999999198, 4.102222222222043, 14.357777777777155)
(18.459999999999848, 3.69199999999997, 14.767999999999878)
(18.46000000000044, 3.3563636363637173, 15.103636363636724)
(18.46000000000002, 3.0766666666666698, 15.383333333333349)
(18.45999999999913, 2.8399999999998666, 15.619999999999264)
(18.460000000000118, 2.6371428571428748, 15.822857142857243)


## Uphill ET Parameter Exploration

Carbon-Limited, NO3, pMMO, Range (-5 - 0) 

In [23]:
for r in np.arange(-5,0,0.2):
    with mcaps:
        mcaps.reactions.CYOR_q8ppi.bounds = r, 1000
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 0
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NO3, pMMO, Uphill ET, {}'.format(r))

Carbon-Limited, NH3, pMMO, Range (-5 - 0)

In [24]:
for r in np.arange(-5,0,0.2):
    with mcaps:
        mcaps.reactions.CYOR_q8ppi.bounds = r, 1000
        mcaps.reactions.EX_nh4_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 0
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NH3, pMMO, Uphill ET, {}'.format(r))

Carbon-Limited, NO3, pMMO, Uphill Unconstrained

In [25]:
with mcaps:
    mcaps.reactions.CYOR_q8ppi.bounds = -1000, 1000
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO, Uphill Unconstrained')

Carbon-Limited, NH4, pMMO, Uphill Unconstrained

In [26]:
with mcaps:
    mcaps.reactions.CYOR_q8ppi.bounds = -1000, 1000
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, Uphill Unconstrained')

In [27]:
df

,"NO3, pMMO, Redox-Arm Only","NO3, sMMO, Redox-Arm Only","NH3, pMMO, Redox-Arm Only","NH3, sMMO, Redox-Arm Only","NO3, pMMO, Redox-Arm, H/ATP 3.0","NO3, pMMO, Redox-Arm, H/ATP 2.8","NO3, pMMO, Redox-Arm, H/ATP 2.6","NO3, pMMO, Redox-Arm, H/ATP 2.4","NO3, pMMO, Redox-Arm, H/ATP 2.2","NO3, pMMO, Redox-Arm, H/ATP 2.0","NO3, pMMO, Redox-Arm, H/ATP 1.8","NO3, pMMO, Redox-Arm, H/ATP 1.6","NO3, pMMO, Redox-Arm, H/ATP 1.4","NO3, pMMO, Redox-Arm, H/ATP 1.2","NO3, pMMODC, RedEff 0.0","NO3, pMMODC, RedEff 0.5","NO3, pMMODC, RedEff 1.0","NO3, pMMODC, RedEff 1.5","NO3, pMMODC, RedEff 2.0","NO3, pMMODC, RedEff 2.5","NO3, pMMODC, RedEff 3.0","NO3, pMMODC, RedEff 3.5","NO3, pMMODC, RedEff 4.0","NO3, pMMODC, RedEff 4.5","NO3, pMMODC, RedEff 5.0","NO3, pMMODC, RedEff 5.5","NO3, pMMODC, RedEff 6.0","NO3, pMMO, Uphill ET, -5.0","NO3, pMMO, Uphill ET, -4.8","NO3, pMMO, Uphill ET, -4.6","NO3, pMMO, Uphill ET, -4.4","NO3, pMMO, Uphill ET, -4.2","NO3, pMMO, Uphill ET, -4.0","NO3, pMMO, Uphill ET, -3.8","NO3, pMMO, Uphill ET, -3.6","NO3, pMMO, Uphill ET, -3.4","NO3, pMMO, Uphill ET, -3.2","NO3, pMMO, Uphill ET, -3.0","NO3, pMMO, Uphill ET, -2.8","NO3, pMMO, Uphill ET, -2.6","NO3, pMMO, Uphill ET, -2.4","NO3, pMMO, Uphill ET, -2.2","NO3, pMMO, Uphill ET, -2.0","NO3, pMMO, Uphill ET, -1.8","NO3, pMMO, Uphill ET, -1.6","NO3, pMMO, Uphill ET, -1.4","NO3, pMMO, Uphill ET, -1.2","NO3, pMMO, Uphill ET, -1.0","NO3, pMMO, Uphill ET, -0.8","NO3, pMMO, Uphill ET, -0.6","NO3, pMMO, Uphill ET, -0.4","NO3, pMMO, Uphill ET, -0.2","NH3, pMMO, Uphill ET, -5.0","NH3, pMMO, Uphill ET, -4.8","NH3, pMMO, Uphill ET, -4.6","NH3, pMMO, Uphill ET, -4.4","NH3, pMMO, Uphill ET, -4.2","NH3, pMMO, Uphill ET, -4.0","NH3, pMMO, Uphill ET, -3.8","NH3, pMMO, Uphill ET, -3.6","NH3, pMMO, Uphill ET, -3.4","NH3, pMMO, Uphill ET, -3.2","NH3, pMMO, Uphill ET, -3.0","NH3, pMMO, Uphill ET, -2.8","NH3, pMMO, Uphill ET, -2.6","NH3, pMMO, Uphill ET, -2.4","NH3, pMMO, Uphill ET, -2.2","NH3, pMMO, Uphill ET, -2.0","NH3, pMMO, Uphill ET, -1.8","NH3, pMMO, Uphill ET, -1.6","NH3, pMMO, Uphill ET, -1.4","NH3, pMMO, Uphill ET, -1.2","NH3, pMMO, Uphill ET, -1.0","NH3, pMMO, Uphill ET, -0.8","NH3, pMMO, Uphill ET, -0.6","NH3, pMMO, Uphill ET, -0.4","NH3, pMMO, Uphill ET, -0.2","NO3, pMMO, Uphill Unconstrained","NH3, pMMO, Uphill Unconstrained"
Methane UTR,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-1.846000e+01,-1.846000e+01,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-1.846000e+01,-1.846000e+01,-1.846000e+01,-1.846000e+01,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000
Oxygen UTR,-27.807105,-27.806966,-27.861020,-27.861020,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-27.806966,-22.265532,-22.869267,-23.250448,-2.416175e+01,-2.476929e+01,-25.203242,-25.528707,-25.781847,-25.984359,-26.150051,-26.288127,-26.404961,-26.505104,-2.533865e+01,-2.543738e+01,-2.553611e+01,-2.563485e+01,-25.733578,-25.832311,-25.931043,-26.029776,-26.128509,-26.227242,-26.325974,-26.424707,-26.523440,-26.622173,-26.720905,-26.819638,-26.918371,-27.017104,-27.115837,-27.214569,-27.313302,-27.412035,-27.510768,-27.609500,-27.708233,-25.407342,-25.505489,-25.603636,-25.701783,-25.799930,-25.898077,-25.996224,-26.094372,-26.192519,-26.290666,-26.388813,-26.486960,-26.585107,-26.683254,-26.781401,-26.879549,-26.977696,-27.

## 'Turning on' the uphill electron transfer

Using the value that was fit above

In [28]:
mcaps.reactions.CYOR_q8ppi.bounds = -2.8, 1000

Carbon-Limited, NO3, sMMO

In [29]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, sMMO')

Carbon-Limited, NO3, pMMO

In [30]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO')

Carbon-Limited, NH3, sMMO

In [31]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, sMMO')

Carbon-Limited, NH3, pMMO

In [32]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO')

Carbon-Limited, N2, sMMO

In [33]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, sMMO')

Carbon-Limited, N2, pMMO

In [34]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, pMMO')

## Direct Coupling 

Carbon-Limited, NO3, pMMODC

In [35]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMODC')

Carbon-Limited, NH3, pMMODC

In [36]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH4, pMMODC')

Carbon-Limited, N2, pMMODC

In [37]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, pMMODC')

## Effect of NH3 oxidation

Carbon-Limited, NH3, pMMO, NO2

In [38]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    c3 = mcaps.problem.Constraint(
    mcaps.reactions.EX_nh4_e.flux_expression + 2* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
    mcaps.add_cons_vars(c3)
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, 1/2*NO2')

Carbon-Limited, NH3, pMMO, NO2, Range(0-5)

In [39]:
np.arange(1,4.2,0.2)

array([1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2, 2.4, 2.6, 2.8, 3. , 3.2, 3.4,
       3.6, 3.8, 4. ])

In [40]:
for r in np.arange(1,4.2,0.2):
    with mcaps:
        mcaps.reactions.EX_nh4_e.bounds = -1000, 0
        mcaps.reactions.PMMOipp.bounds = 0, 1000
        c3 = mcaps.problem.Constraint(
        mcaps.reactions.EX_nh4_e.flux_expression + r* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
        mcaps.add_cons_vars(c3)
        mcaps.reactions.GLNS.bounds = 0, 0
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NH3, pMMO, 1/{}*NO2'.format(r))

Carbon-Limited, NO3, pMMO, NO2

In [41]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    c3 = mcaps.problem.Constraint(
    mcaps.reactions.EX_nh4_e.flux_expression + 2* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
    mcaps.add_cons_vars(c3)
    mcaps.reactions.GLNS.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO, 1/2*NO2')

O2-Limited, NH3, pMMO, 0.29

In [42]:
with mcaps:
    mcaps.objective = mcaps.reactions.EX_o2_e
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 0
    mcaps.reactions.BIOMASS_Mcapsulatus.bounds = 0.291, 0.291
    solution = pfba(mcaps)

df = df_record(solution, df, 'O2-Limited, NH3, pMMO, 0.29')

C-Limited, NO3, pMMO, KOTerminalOxidase

In [43]:
# mcaps.reactions.CYTAA3ipp

In [44]:
# with mcaps:
#     mcaps.reactions.EX_nh4_e.bounds = -1000, 0
#     mcaps.reactions.PMMOipp.bounds = 0, 1000
#     mcaps.reactions.GLNS.bounds = 0, 1000
#     mcaps.reactions.CYTAA3ipp.bounds = 0, 0
#     mcaps.reactions.CYTCBDipp.bounds = 0, 0
#     solution = pfba(mcaps)

# df = df_record(solution, df, 'NO3, pMMO, KOTerminalOxidase')

In [45]:
df.round(2)

,"NO3, pMMO, Redox-Arm Only","NO3, sMMO, Redox-Arm Only","NH3, pMMO, Redox-Arm Only","NH3, sMMO, Redox-Arm Only","NO3, pMMO, Redox-Arm, H/ATP 3.0","NO3, pMMO, Redox-Arm, H/ATP 2.8","NO3, pMMO, Redox-Arm, H/ATP 2.6","NO3, pMMO, Redox-Arm, H/ATP 2.4","NO3, pMMO, Redox-Arm, H/ATP 2.2","NO3, pMMO, Redox-Arm, H/ATP 2.0","NO3, pMMO, Redox-Arm, H/ATP 1.8","NO3, pMMO, Redox-Arm, H/ATP 1.6","NO3, pMMO, Redox-Arm, H/ATP 1.4","NO3, pMMO, Redox-Arm, H/ATP 1.2","NO3, pMMODC, RedEff 0.0","NO3, pMMODC, RedEff 0.5","NO3, pMMODC, RedEff 1.0","NO3, pMMODC, RedEff 1.5","NO3, pMMODC, RedEff 2.0","NO3, pMMODC, RedEff 2.5","NO3, pMMODC, RedEff 3.0","NO3, pMMODC, RedEff 3.5","NO3, pMMODC, RedEff 4.0","NO3, pMMODC, RedEff 4.5","NO3, pMMODC, RedEff 5.0","NO3, pMMODC, RedEff 5.5","NO3, pMMODC, RedEff 6.0","NO3, pMMO, Uphill ET, -5.0","NO3, pMMO, Uphill ET, -4.8","NO3, pMMO, Uphill ET, -4.6","NO3, pMMO, Uphill ET, -4.4","NO3, pMMO, Uphill ET, -4.2","NO3, pMMO, Uphill ET, -4.0","NO3, pMMO, Uphill ET, -3.8","NO3, pMMO, Uphill ET, -3.6","NO3, pMMO, Uphill ET, -3.4","NO3, pMMO, Uphill ET, -3.2","NO3, pMMO, Uphill ET, -3.0","NO3, pMMO, Uphill ET, -2.8","NO3, pMMO, Uphill ET, -2.6","NO3, pMMO, Uphill ET, -2.4","NO3, pMMO, Uphill ET, -2.2","NO3, pMMO, Uphill ET, -2.0","NO3, pMMO, Uphill ET, -1.8","NO3, pMMO, Uphill ET, -1.6","NO3, pMMO, Uphill ET, -1.4","NO3, pMMO, Uphill ET, -1.2","NO3, pMMO, Uphill ET, -1.0","NO3, pMMO, Uphill ET, -0.8","NO3, pMMO, Uphill ET, -0.6","NO3, pMMO, Uphill ET, -0.4","NO3, pMMO, Uphill ET, -0.2","NH3, pMMO, Uphill ET, -5.0","NH3, pMMO, Uphill ET, -4.8","NH3, pMMO, Uphill ET, -4.6","NH3, pMMO, Uphill ET, -4.4","NH3, pMMO, Uphill ET, -4.2","NH3, pMMO, Uphill ET, -4.0","NH3, pMMO, Uphill ET, -3.8","NH3, pMMO, Uphill ET, -3.6","NH3, pMMO, Uphill ET, -3.4","NH3, pMMO, Uphill ET, -3.2","NH3, pMMO, Uphill ET, -3.0","NH3, pMMO, Uphill ET, -2.8","NH3, pMMO, Uphill ET, -2.6","NH3, pMMO, Uphill ET, -2.4","NH3, pMMO, Uphill ET, -2.2","NH3, pMMO, Uphill ET, -2.0","NH3, pMMO, Uphill ET, -1.8","NH3, pMMO, Uphill ET, -1.6","NH3, pMMO, Uphill ET, -1.4","NH3, pMMO, Uphill ET, -1.2","NH3, pMMO, Uphill ET, -1.0","NH3, pMMO, Uphill ET, -0.8","NH3, pMMO, Uphill ET, -0.6","NH3, pMMO, Uphill ET, -0.4","NH3, pMMO, Uphill ET, -0.2","NO3, pMMO, Uphill Unconstrained","NH3, pMMO, Uphill Unconstrained","NO3, sMMO","NO3, pMMO","NH3, sMMO","NH3, pMMO","N2, sMMO","N2, pMMO","NO3, pMMODC","NH4, pMMODC","N2, pMMODC","NH3, pMMO, 1/2*NO2","NH3, pMMO, 1/1.0*NO2","NH3, pMMO, 1/1.2*NO2","NH3, pMMO, 1/1.4*NO2","NH3, pMMO, 1/1.6*NO2","NH3, pMMO, 1/1.8*NO2","NH3, pMMO, 1/2.0*NO2","NH3, pMMO, 1/2.2*NO2","NH3, pMMO, 1/2.4*NO2","NH3, pMMO, 1/2.6*NO2","NH3, pMMO, 1/2.8*NO2","NH3, pMMO, 1/3.0*NO2","NH3, pMMO, 1/3.2*NO2","NH3, pMMO, 1/3.4*NO2","NH3, pMMO, 1/3.6*NO2","NH3, pMMO, 1/3.8*NO2","NH3, pMMO, 1/4.0*NO2","NO3, pMMO, 1/2*NO2","O2-Limited, NH3, pMMO, 0.29"
Methane UTR,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-2.58,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-17.84
Oxygen UTR,-27.81,-27.81,-27.86,-27.86,-27.81,-27.81,-27.81,-27.81,-27.81,-27.81,-27.81,-27.81,-27.81,-27.81,-22.27,-22.87,-23.25,-24.16,-24.77,-25.20,-25.53,-25.78,-25.98,-26.15,-26.29,-26.40,-26.51,-25.34,-25.44,-25.54,-25.63,-25.73,-25.83,-25.93,-26.03,-26.13,-26.23,-26.33,-26.42,-26.52,-26.62,-26.72,-26.82,-26.92,-27.02,-27.12,-27.21,-27.31,-27.41,-27.51,-27.61,-27.71,-25.4

In [157]:
df.to_csv("../ComplementaryData/SupplementTable2.csv")

In [56]:
import matplotlib.pyplot as plt

In [57]:
plt.plot([1,2,3,4])

In [58]:
plt.draw()

In [59]:
plt.show()

In [60]:
import matplotlib.pyplot as plt

In [61]:
plt.figure(figsize=(5,3), dpi=300);

df[['NO3, sMMO','NO3, pMMO','NH3, sMMO','NH3, pMMO','N2, sMMO','N2, pMMO']].round(2).loc['Growth Yield'].plot.bar(); plt.axhline(0, color='k')

In [62]:
df[['NO3, sMMO','NO3, pMMO','NH3, sMMO','NH3, pMMO','N2, sMMO','N2, pMMO']]

,"NO3, sMMO","NO3, pMMO","NH3, sMMO","NH3, pMMO","N2, sMMO","N2, pMMO"
Methane UTR,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000
Oxygen UTR,-27.025099,-26.424707,-27.861020,-26.486960,-28.782341,-26.772992
CO2 UTR,11.954014,11.559251,9.699031,8.370173,11.182966,9.386123
NH4 UTR,0.000000,0.000000,-2.132753,-2.456248,0.000000,0.000000
N2 UTR,0.000000,0.000000,0.000000,0.000000,-0.600096,-0.748272
NO3 UTR,-1.583804,-1.679905,0.000000,0.000000,0.000000,0.000000
Uphill Electron Transfer Flux,-1.583804,-2.800000,0.000000,-2.800000,1.409985,-2.800000
Growth Rate,0.193276,0.205004,0.260266,0.299743,0.081258,0.101322
Growth Yield,0.652744,0.692350,0.878985,1.012309,0.274428,0.342189
O2/CH4 Ratio,1.463982,1.431458,1.509264,1.434830,1.559173,1.450325


In [63]:
plt.figure(figsize=(5,3), dpi=300);

df[['NO3, sMMO','NO3, pMMO','NH3, sMMO','NH3, pMMO','N2, sMMO','N2, pMMO']].round(2).loc['O2/CH4 Ratio'].plot.bar(ylim=(1.1,1.7)); plt.axhline(0, color='k')

In [64]:
plt.figure(figsize=(5,3), dpi=300);

df[['NO3, pMMODC','NH4, pMMODC','N2, pMMODC','NH3, pMMO, 1/2*NO2']].loc['O2/CH4 Ratio'].plot.bar(ylim=(1.1,1.7))

### Replication of Table 1 by Leak&Dalton

In [46]:
df2 = pd.DataFrame(index=["Methane UTR","Oxygen UTR", "CO2 UTR", "NH4 UTR", "N2 UTR",
                         "NO3 UTR", "Uphill Electron Transfer Flux", "Growth Rate" ,
                         "Growth Yield", "O2/CH4 Ratio", "NH4/CH4 Ratio", "N2/CH4 Ratio",
                         "NO3/CH4 Ratio", "ATP/CH4 Ratio", "CCE"])

In [50]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    solution = pfba(mcaps)

df2 = df_record(solution, df2, 'NO3, sMMO')

In [51]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df2 = df_record(solution, df2, 'NO3, pMMO')

In [53]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    solution = pfba(mcaps)

df2 = df_record(solution, df2, 'NH3, sMMO')

In [54]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df2 = df_record(solution, df2, 'NH3, pMMO')

In [55]:
df2

,"NO3, sMMO","NO3, pMMO","NH3, sMMO","NH3, pMMO"
Methane UTR,-18.460000,-18.460000,-18.460000,-18.460000
Oxygen UTR,-27.806966,-26.424707,-27.861020,-26.486960
CO2 UTR,12.468098,11.559251,9.699031,8.370173
NH4 UTR,0.000000,0.000000,-2.132753,-2.456248
N2 UTR,0.000000,0.000000,0.000000,0.000000
NO3 UTR,-1.458657,-1.679905,0.000000,0.000000
Uphill Electron Transfer Flux,0.000000,-2.800000,0.000000,-2.800000
Growth Rate,0.178004,0.205004,0.260266,0.299743
Growth Yield,0.601166,0.692350,0.878985,1.012309
O2/CH4 Ratio,1.506336,1.431458,1.509264,1.434830


In [64]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    mcaps.add_boundary(mcaps.metabolites.ac_c, lb=-1000)
    solution = pfba(mcaps)

df2 = df_record(solution, df2, 'NO3, Acetate')

Infeasible: None

In [65]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    mcaps.add_boundary(mcaps.metabolites.etoh_c, lb=-1000)
    solution = pfba(mcaps)

df2 = df_record(solution, df2, 'NO3, EtOH')

/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:4 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:7 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:8 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:9 RuntimeWarning: divide by zero encountered in double_scalars


In [73]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 1000
    mcaps.reactions.SMMOi.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    mcaps.reactions.EX_fald_e.bounds = -1000, 1000
    solution = pfba(mcaps)

df2 = df_record(solution, df2, 'NO3, Fald')

/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:4 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:7 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:7 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:8 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:8 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:9 RuntimeWarning: divide by zero encountered in double_scalars


In [76]:
mcaps.medium

{'EX_ca2_e': 1000.0,
 'EX_cd2_e': 1000.0,
 'EX_ch4_e': 18.46,
 'EX_cl_e': 1000.0,
 'EX_co_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [85]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 1000
    mcaps.reactions.SMMOi.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    mcaps.reactions.EX_co2_e.bounds = -1000, 1000
    solution = pfba(mcaps)
    print solution.fluxes.BIOMASS_Mcapsulatus

df2 = df_record(solution, df2, 'NO3, CO2')

Infeasible: None

In [83]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
    mcaps.reactions.EX_meoh_e.bounds = -1000, 1000
    solution = pfba(mcaps)
    print solution.fluxes.BIOMASS_Mcapsulatus
    
df2 = df_record(solution, df2, 'NO3, Methanol')

9.642695794430523


/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:4 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:7 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:7 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:8 RuntimeWarning: invalid value encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:8 RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/envs/efpro2/lib/python2.7/site-packages/ipykernel_launcher.py:9 RuntimeWarning: divide by zero encountered in double_scalars


In [86]:
df2

,"NO3, sMMO","NO3, pMMO","NH3, sMMO","NH3, pMMO","NO3, Acetate","NO3, EtOH","NO3, Fald","NO3, Methanol"
Methane UTR,-18.460000,-18.460000,-18.460000,-18.460000,0.000000e+00,0.000000,0.000000,0.000000
Oxygen UTR,-27.806966,-26.424707,-27.861020,-26.486960,-1.864444e+00,-118.610658,-118.610658,-256.336169
CO2 UTR,12.468098,11.559251,9.699031,8.370173,0.000000e+00,420.478595,420.478595,175.411599
NH4 UTR,0.000000,0.000000,-2.132753,-2.456248,2.329705e-16,0.000000,0.000000,0.000000
N2 UTR,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
NO3 UTR,-1.458657,-1.679905,0.000000,0.000000,0.000000e+00,-141.077544,-141.077544,-79.017158
Uphill Electron Transfer Flux,0.000000,-2.800000,0.000000,-2.800000,3.728889e+00,214.596074,214.596074,0.000000
Growth Rate,0.178004,0.205004,0.260266,0.299743,6.482567e-17,17.216107,17.216107,9.642696
Growth Yield,0.601166,0.692350,0.878985,1.012309,inf,inf,inf,inf
O2/CH4 Ratio,1.506336,1.431458,1.509264,1.434830,-inf,-inf,-inf,-inf
